In [13]:
import tensorflow.keras as keras
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.utils import shuffle, class_weight
import pandas as pd
import numpy as np
import tensorflow as tf

In [14]:
path = r"Overall_DA_Dataset_with_APR.csv"
df = pd.read_csv(path)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78711 entries, 0 to 78710
Columns: 105 entries, x_mean to Labels
dtypes: float64(86), int64(18), object(1)
memory usage: 63.1+ MB


In [16]:
df.Labels.value_counts()

Asphalt_Ride        51189
Stop                14193
Cobblestone_Ride     9508
Asphalt_Manhole      2824
Asphalt_Kurb          621
Asphalt_Bump          376
Name: Labels, dtype: int64

In [17]:
class_weight = {0: 1.,
                1: 3.,
                2: 5.,
               3: 22.,
               4: 80.,
               5: 130.,
               }

In [18]:
N_train_samples = 70000
N_test_samples = 8711
N_samples = N_train_samples + N_test_samples
N_epochs = 100
batch_size = 64
learning_rate = 0.01

In [19]:
X = df.drop('Labels', axis=1)
y = df["Labels"]

In [20]:
y = np.expand_dims(y, axis=-1)

# encode class values as integers
encoder = OneHotEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y = encoded_y.todense()

In [21]:
shuffled_x, shuffled_y= shuffle(X, encoded_y)

In [22]:
x_train = shuffled_x[0:N_train_samples]
y_train = shuffled_y[0:N_train_samples]

x_test = shuffled_x[N_train_samples: N_train_samples+N_test_samples]
y_test = shuffled_y[N_train_samples: N_train_samples+N_test_samples]

In [23]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size, drop_remainder=True).shuffle(1000).repeat()

In [24]:
#x_train, x_test, y_train, y_test = train_test_split(X.values, encoded_y, test_size=0.1)

In [25]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(101, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(6, activation='softmax'))

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_ds,  batch_size=batch_size, verbose=1, epochs=N_epochs, steps_per_epoch = N_samples/batch_size, class_weight = class_weight)

Epoch 1/100
1229/1229 [==============================] - 3s 2ms/step - loss: 24.5440 - accuracy: 0.7378
Epoch 2/100
1229/1229 [==============================] - 2s 2ms/step - loss: 11.7158 - accuracy: 0.8347
Epoch 3/100
1229/1229 [==============================] - 3s 2ms/step - loss: 9.0160 - accuracy: 0.8638
Epoch 4/100
1229/1229 [==============================] - 3s 2ms/step - loss: 8.2138 - accuracy: 0.8738
Epoch 5/100
1229/1229 [==============================] - 3s 2ms/step - loss: 7.4782 - accuracy: 0.8829
Epoch 6/100
1229/1229 [==============================] - 3s 2ms/step - loss: 7.0001 - accuracy: 0.8859
Epoch 7/100
1229/1229 [==============================] - 3s 2ms/step - loss: 6.6509 - accuracy: 0.8889
Epoch 8/100
1229/1229 [==============================] - 3s 2ms/step - loss: 6.3732 - accuracy: 0.8909
Epoch 9/100
1229/1229 [==============================] - 3s 2ms/step - loss: 5.9428 - accuracy: 0.8972
Epoch 10/100
1229/1229 [==============================] - 3s 2ms/step -

1229/1229 [==============================] - 3s 2ms/step - loss: 3.3230 - accuracy: 0.9270
Epoch 80/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.1651 - accuracy: 0.9289
Epoch 81/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.4494 - accuracy: 0.9263
Epoch 82/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.3361 - accuracy: 0.9275
Epoch 83/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.1784 - accuracy: 0.9287
Epoch 84/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.2231 - accuracy: 0.9276
Epoch 85/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.2611 - accuracy: 0.9272
Epoch 86/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.2252 - accuracy: 0.9282
Epoch 87/100
1229/1229 [==============================] - 3s 2ms/step - loss: 3.1784 - accuracy: 0.9286
Epoch 88/100
1229/1229 [==============================] - 3s 2ms/step - loss:

In [27]:
encoder.categories_[0]

array(['Asphalt_Bump', 'Asphalt_Kurb', 'Asphalt_Manhole', 'Asphalt_Ride',
       'Cobblestone_Ride', 'Stop'], dtype=object)

In [28]:
y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_test_class = np.argmax(y_test, axis=1)
confusion_matrix(y_test_class, y_pred_class)

273/273 [==============================] - 1s 1ms/step


array([[   0,    0,   11,   21,    5,    0],
       [   1,    2,    8,   35,   26,    0],
       [   0,    0,   22,  243,   44,    0],
       [   0,    0,    0, 5397,  166,   33],
       [   0,    0,    0,   11, 1044,    6],
       [   0,    0,    0,    9,    4, 1623]], dtype=int64)

In [29]:
nn_f1_s = f1_score(y_test_class, y_pred_class, average ="weighted")
nn_f1_s

0.9109631375128984

In [30]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        37
           1       1.00      0.03      0.05        72
           2       0.54      0.07      0.13       309
           3       0.94      0.96      0.95      5596
           4       0.81      0.98      0.89      1061
           5       0.98      0.99      0.98      1636

    accuracy                           0.93      8711
   macro avg       0.71      0.51      0.50      8711
weighted avg       0.92      0.93      0.91      8711



In [31]:
model.save(r"C:\Users\arockias\Desktop\SmartRadL\SmartRadL\Programs\Machine Learning\Models\NN_model")

INFO:tensorflow:Assets written to: C:\Users\arockias\Desktop\SmartRadL\SmartRadL\Programs\Machine Learning\Models\NN_model\assets
